In [1]:
import re
import json
import nltk
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from tqdm import tqdm

### Preprocess

In [111]:
# CHECK BEFORE RUNNING!!!
flag_preprocess = False # If false, skip the generation of a preprocessed corpus
flag_cluster_labels = False 
flag_cluster = False # If false, stop overwrite of a cluster matrix

In [3]:
# Define column names for movie.metadata.tsv
movie_metadata_columns = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie name", "Movie release date",
                          "Movie box office revenue", "Movie runtime", "Movie languages", "Movie countries",
                          "Movie genres"]

# Read movie.metadata.tsv into a DataFrame
movie_metadata_df = pd.read_csv("data/movie.metadata.tsv", sep='\t', header=None, names=movie_metadata_columns)

In [4]:
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', ' ', text)
    # Convert to lowercase and tokenize
    words = nltk.word_tokenize(text.lower())
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

# Open 'plot_summaries.txt'
with open('data/plot_summaries.txt', 'r', encoding='utf-8') as file:
    summaries = file.readlines()

# Extract movie IDs 
movie_ids = [int(summary.split()[0]) for summary in summaries]

# Preprocess the summaries 
preprocessed_summaries = []
if flag_preprocess:
    for summary in tqdm(summaries, desc="Processing summaries"):
        preprocessed_summaries.append(preprocess_text(summary.split('\t')[1]))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mathiaskroismoller/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mathiaskroismoller/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Save preprocessed summaries to a pkl file
import csv
if flag_preprocess:
    # Save preprocessed_summaries list to a file
    with open('preprocessed_summaries.pkl', 'wb') as file:
        pickle.dump(preprocessed_summaries, file)

In [6]:
# Combined stemmed summaries with labels 
summaries_preprocessed_dist = []
if flag_preprocess:
    for i in range(len(preprocessed_summaries)):
        summaries_preprocessed_dist.append({"id": movie_ids[i], "text": preprocessed_summaries[i]})
    
    # Save summaries_preprocessed_dist to a JSON file
    with open('summaries_preprocessed_dist.json', 'w') as f:
        json.dump(summaries_preprocessed_dist, f)

# Home made version TF/IDF vectorization

In [7]:
# Load preprocessed_summaries list from the saved file
with open('preprocessed_summaries.pkl', 'rb') as file:
    preprocessed_summaries = pickle.load(file)

In [8]:
# Convert text data into numerical features using TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_summaries)


In [11]:
# Apply KMeans clustering
if flag_cluster_labels:
    num_clusters = 100  # You can adjust the number of clusters -> the more clusters the heavier the computation
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(tfidf_matrix) # This takes a loooong time...
    
    # Save cluster_labels to a file
    with open('cluster_labels.pkl', 'wb') as f:
        pickle.dump(cluster_labels, f)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [26]:
# Load cluster_labels from a file
with open('cluster_labels.pkl', 'rb') as f:
    cluster_labels = pickle.load(f)

In [106]:
# Create a dictionary to store movie IDs and their corresponding cluster labels
movie_clusters = dict(zip(movie_ids, cluster_labels))

# Save the dictionary to a file using pickle
with open('movie_clusters_id.pkl', 'wb') as file:
    pickle.dump(movie_clusters, file)

In [107]:
# Create a DataFrame to store cluster information
cluster_df = pd.DataFrame(columns=[f'Cluster {i}' for i in range(num_clusters)])

# Iterate through movie clusters and match with movie_metadata_df
if flag_cluster:
    for movie_id, cluster_label in tqdm(movie_clusters.items()):
        # Find the corresponding row in movie_metadata_df based on Wikipedia Movie ID
        movie_row = movie_metadata_df[movie_metadata_df['Wikipedia Movie ID'] == movie_id]
        
        # Extract movie name from the row
        movie_name = movie_row['Movie name'].values[0] if not movie_row.empty else f'Movie {movie_id}'
        
        # Update cluster_df with movie name in the appropriate cluster column
        cluster_df.loc[len(cluster_df), f'Cluster {cluster_label}'] = movie_name
    
    
    # Fill NaN values with empty string for better representation
    cluster_df.fillna('', inplace=True)

100%|██████████| 42306/42306 [20:07<00:00, 35.04it/s]  


In [110]:
# Save cluster_df as CSV
if flag_cluster:
    cluster_df.to_csv('clustered_movies.csv', index=True)

In [82]:
# Load cluster_df from a file
cluster_df = pd.read_csv('clustered_movies.csv')

/var/folders/hg/tvll5hmx5b580xjtn0kmrff00000gn/T/ipykernel_5219/4017522519.py:2: DtypeWarning: Columns (31,50) have mixed types. Specify dtype option on import or set low_memory=False.
  cluster_df = pd.read_csv('clustered_movies.csv')


### Top 10 words pr cluster

In [0]:
def get_top_words_for_clusters(tfidf_matrix, cluster_labels, vectorizer, top_n=10):
    cluster_words = {}
    terms = vectorizer.get_feature_names_out()
    
    for cluster_label in range(max(cluster_labels) + 1):
        cluster_indices = np.where(cluster_labels == cluster_label)[0]
        cluster_tfidf_scores = np.sum(tfidf_matrix[cluster_indices], axis=0)
        sorted_indices = np.argsort(cluster_tfidf_scores)[0, ::-1][:top_n]
        top_words = [terms[i] for i in sorted_indices]
        cluster_words[cluster_label] = top_words
        
    return cluster_words

# Call the function to get cluster words
cluster_words = get_top_words_for_clusters(tfidf_matrix, cluster_labels, vectorizer)

In [63]:
cluster_words[0][0][0][:10]

array(['peter', 'love', 'man', 'wendy', 'spider', 'father', 'jack',
       'back', 'home', 'one'], dtype=object)

In [71]:
# Dataframe for top 1000 words ranked for every cluster
cluster_words_ranked = pd.DataFrame()

for i in range(len(cluster_words)):
    cluster_words_ranked[f'{i}'] = cluster_words[i][0][0][:1000]
    
cluster_words_ranked

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,peter,jesse,raju,school,henry,cite,agent,bill,one,love,...,vijay,jim,charles,joan,man,sylvester,harry,martin,bob,creature
1,love,james,karan,students,anne,web,president,new,police,falls,...,love,silver,katie,joseph,young,tweety,one,police,larry,monster
2,man,willy,amar,high,catherine,synopsis,united,life,two,marriage,...,ravi,father,edward,jack,old,cat,voldemort,wife,kevin,godzilla
3,wendy,ethan,vicky,teacher,sir,film,states,connie,life,girl,...,kumar,new,bella,life,one,granny,two,family,song,frankenstein
4,spider,celine,love,student,family,based,soviet,film,new,daughter,...,father,stifler,sir,father,wife,speedy,man,blomkvist,however,dragon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,staff,moving,sell,yuko,shakespeare,classic,opens,ever,youth,aristocrat,...,birth,cop,ceremony,encounter,jealous,vacation,feet,words,detective,close
996,lomax,radio,koya,post,accepts,sik,especially,honesty,pass,satya,...,indeed,eye,quickly,famine,dave,exhibit,pc,engineer,tree,cold
997,mcnally,pull,millionaire,reluctant,befriends,murderer,married,goldie,program,present,...,motley,protecting,neighbor,dinner,action,holding,elevator,sexual,search,liquid
998,nurse,shadow,microdot,naomi,realize,train,interviews,jackie,miller,american,...,kallu,suddenly,health,bloodthirsty,stage,mission,states,highwaymen,rob,weapons


In [72]:
cluster_words_ranked.to_csv('cluster_words.csv', index=False, header=True)

# NPL version (Nothing yet)

In [85]:
# Load summaries_preprocessed_dist from a JSON file
with open('summaries_preprocessed_dist.json', 'r') as f:
    summaries_preprocessed_dist = json.load(f)

In [63]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [64]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [86]:
tokenized_sum = summaries_preprocessed_dist.map(preprocess_function, batched=True)

AttributeError: 'list' object has no attribute 'map'

# Enriching data
#### 1. Adding data from www.themoviedb.org (TMDB)

In this section we are going to be using the API of www.themoviedb.org (TMDB) to populate the existing dataframe with more information about the movie. The script is going to query the website with the movie name and match with the movie, where the release date is the closest. If no match is found the code skips the movie and goes to the next.

This 1. step is most importantly adding the TMDB_ID which is a unique identifier for the movie which can be used to further enrich the data.

This script was run separately in another file in /scratch/add_TMDB_movie_metadata.py as it needed to be run several times since the API was limited by 40 requests/s.
 


In [10]:
import tmdbsimple as tmdb # Wrapper library for the API of themoviedb.org  (TMDB)
from tqdm import tqdm # Progress bar for the script
import pandas as pd
from datetime import datetime # Used for comparing movie release dates
from dotenv import load_dotenv # Makes keeping the API-key as local environment file simpler
import os # Used for loading the .env file

load_dotenv() # Loads .env files aka. the TMDB_API_KEY

headers_movie_metadata = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie name", "Movie release date",
                          "Movie box office revenue", "Movie runtime", "Movie languages", "Movie countries",
                          "Movie genres"]
movie_metadata = pd.read_csv('/data/movie.metadata.tsv', sep="\t", names=headers_movie_metadata)


# Load API key
TMDB_API_KEY = os.environ.get("TMDB_API_KEY")
tmdb.API_KEY = TMDB_API_KEY
tmdb.REQUESTS_TIMEOUT = 5  # Seconds, for both connect and read

# Create a list to save progress
saved_progress = []

# Determine where to resume
start_index = 28000

# Progress file that can be used to resume
#saved_progress = pd.read_json('progress.json')['index'].tolist()
#start_index = saved_progress[-1] + 1  # Start from the next index

# Create a DataFrame to store the data
movie_metadata_TMDB = movie_metadata.copy()

for index, row in tqdm(movie_metadata_TMDB.iterrows(), total=len(movie_metadata_TMDB), desc="Processing"): # Wraps for loop in progress bar.
    
    # Skip previously processed indices - Commented out in favor of manual start_index
    #if index in saved_progress:
    #    continue
    if index < start_index:
        continue
    try:
        if not pd.isna(row["Movie release date"]):
            search = tmdb.Search()
            response = search.movie(query=row["Movie name"])

            # Convert dataframe release date to datetime
            movie_release_date_str = row["Movie release date"]
            if len(movie_release_date_str) == 4:  # Handle "YYYY" format
                movie_release_date = datetime.strptime(movie_release_date_str, "%Y").date()
            elif len(movie_release_date_str) == 7:  # Handle "YYYY-DD" format
                movie_release_date = datetime.strptime(movie_release_date_str, "%Y-%m").date()
            else:  # Assume it's in the format "YYYY-MM-DD"
                movie_release_date = datetime.strptime(movie_release_date_str, "%Y-%m-%d").date()

            # Handle if release_date is empty.
            date_list_converted = [datetime.strptime(each_date['release_date'], "%Y-%m-%d").date() for each_date in
                                   search.results if each_date.get('release_date')]

            # Create list of differences in time
            differences = [abs(movie_release_date - each_date) for each_date in date_list_converted]
            
            # If differences are empty = skip
            if not differences:
                continue
            minimum_index = differences.index(min(differences))  # Index of the closest match
            match = search.results[minimum_index]
            # print(f"Closest match: {match['title']} (Release Date: {match['release_date']})")

            # Add info in dataframe about the movie
            movie_metadata_TMDB.loc[index, 'TMDB_id'] = match['id']
            movie_metadata_TMDB.loc[index, 'TMDB_original_language'] = match['original_language']
            movie_metadata_TMDB.loc[index, 'TMDB_original_title'] = match['original_title']
            movie_metadata_TMDB.loc[index, 'TMDB_overview'] = match['overview']
            movie_metadata_TMDB.loc[index, 'TMDB_popularity'] = match['popularity']
            movie_metadata_TMDB.loc[index, 'TMDB_release_date'] = match['release_date']
            movie_metadata_TMDB.loc[index, 'TMDB_title'] = match['title']
            movie_metadata_TMDB.loc[index, 'TMDB_vote_average'] = match['vote_average']
            movie_metadata_TMDB.loc[index, 'TMDB_vote_count'] = match['vote_count']
            
            # Save the index as progress
            saved_progress.append(index)

            # Save progress periodically (in case of interruption)
            if index % 50 == 0:
                progress_df = pd.DataFrame({'index': saved_progress})
                progress_df.to_json('progress.json')
                movie_metadata_TMDB.to_csv('movie_metadata_TMDB.csv', index=False)

    except Exception as e:
        print(f"Error at index {index}: {str(e)}")

# Save final progress
progress_df = pd.DataFrame({'index': saved_progress})
progress_df.to_json('progress.json')

# Save your final DataFrame
movie_metadata_TMDB.to_csv('modified_data/movie_metadata_TMDB.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/data/movie.metadata.tsv'

In [3]:
# Comparing how many rows were populated in the enriching.
import pandas as pd
df = pd.read_csv('modified_data/movie_metadata_TMDB.csv')
print(f"Added TMDB ID to {df['TMDB_id'].count()} movies. Total movies: {df['Wikipedia Movie ID'].count()}. \n Percentage populated: {round(df['TMDB_id'].count()/df['Wikipedia Movie ID'].count()*100,2)}%")


Added TMDB ID to 68944 movies. Total movies: 81741. 
 Percentage populated: 84.34%


#### 2. Adding Movie Revenue
Many of the movies are missing their revenue and TMDB has that information.
E.g.: https://www.themoviedb.org/movie/51284